# Purpose of the notebook

In this notebook we explore the results of different preprocessing strategies on simulated data

## Loading the packages 

In [10]:
# import txsim as tx
import scanpy as sc
import os
import pandas as pd
from anndata import AnnData
import numpy as np
import xb.formatting as xf
import seaborn as sns
import random 
import matplotlib.pyplot as plt
import sklearn.metrics as sk
from tqdm import tqdm
import scanpy as sc
import random
from xb.calculating import entropy,compute_vi,compute_fmi
from xb.formatting import keep_nuclei_and_quality
from xb.simulating import missegmentation_simulation,noise_adder,subset_of_single_cell,main_preprocessing, allcombs_simulated

## Specify path to scRNAseq simulations

In [11]:
mainpath='../../data/scRNAseq_for_simulations/'
datasets=os.listdir(mainpath)
datasets=[d for d in datasets if d!='.ipynb_checkpoints']
rsimpath='../../data/scRNAseq_for_simulations_Xenium_like/'

We now extract clustering results from both SEURAT AND SCANPY's preprocessing strategies

In [12]:
datasetdic={}
dats=[]
available_tiss=[]
for d in range(0,len(datasets)):
    resul=pd.read_csv(mainpath+datasets[d]+'/final_allresults.csv',index_col=0)
    rres=pd.read_csv(rsimpath+datasets[d]+'/results_clustering.csv',index_col=0)
    resul.columns='SCANPY_'+resul.columns
    rres.columns='SEURAT_'+rres.columns
    adata=sc.read(mainpath+datasets[d]+'/original_adata.h5ad')
    if adata.obs['tissue'][0] not in available_tiss:
        datasetdic[datasets[d]]=adata.obs['tissue'][0]
        available_tiss.append(adata.obs['tissue'][0])
    else:
        tot=np.sum([adata.obs['tissue'][0] in n for n in available_tiss])
        datasetdic[datasets[d]]=adata.obs['tissue'][0]+'_'+str(tot)
        available_tiss.append(adata.obs['tissue'][0]+'_'+str(tot))


FileNotFoundError: [Errno 2] No such file or directory: '../../data/scRNAseq_for_simulations_Xenium_like/9968be68-ab65-4a38-9e1a-c9b6abece194/results_clustering.csv'

# Extract all simulation conditions

In [4]:
allpos=[]
for d in range(0,len(datasets)):
    try:
        resul=pd.read_csv(mainpath+datasets[d]+'/final_allresults.csv',index_col=0)
        rres=pd.read_csv(rsimpath+datasets[d]+'/results_clustering.csv',index_col=0)
        resul.columns='SCANPY_'+resul.columns
        rres.columns='SEURAT_'+rres.columns
        for el in resul.columns:
            allpos.append(el)
        for el in rres.columns:
            allpos.append(el)
    except:
        E=2

After this, we define dataframes where we will store the ARI, VI and FMI scores for each simulation on each dataset

In [5]:
ARI=None
VI=None
fmi=None
if ARI is None:
    ARI=pd.DataFrame(index=np.unique(allpos),columns=datasets)
if VI is None:
    VI=pd.DataFrame(index=np.unique(allpos),columns=datasets)
if fmi is None:
    fmi=pd.DataFrame(index=np.unique(allpos),columns=datasets)

And we compute these scores for the datasets

In [6]:
dats=[]
from tqdm import tqdm
for d in tqdm(range(0,len(datasets))):
    try:
        resul=pd.read_csv(mainpath+datasets[d]+'/final_allresults.csv',index_col=0)
        rres=pd.read_csv(rsimpath+datasets[d]+'/results_clustering.csv',index_col=0)
        resul.columns='SCANPY_'+resul.columns
        rres.columns='SEURAT_'+rres.columns
        dats.append(datasets[d])
        for ind in resul.columns:
            ARI.loc[ind,datasets[d]]=sk.adjusted_rand_score(resul.loc[:,ind],resul.loc[:,'SCANPY_RESULTS'])
        for ind in rres.columns:
            ARI.loc[ind,datasets[d]]=sk.adjusted_rand_score(rres.loc[:,ind],resul.loc[:,'SCANPY_RESULTS'])
    except:
        ei=2

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [00:00<00:00, 26325.09it/s]


# Plotting the results 

We first format and plot the ARI

In [7]:
res=[el.split('_')[0] for el in ARI.index]
ARISEL=ARI.loc[:,dats].fillna(0)
ARISEL=ARISEL.loc[np.sum(ARISEL.fillna(-1)>0,axis=1)>30,:]
ARISEL=ARISEL.loc[~ARISEL.index.isin(['SCANPY_RESULTS']),:]
ARISEL_mean=pd.DataFrame(np.mean(ARISEL,axis=1)).sort_values(by=0)
ARISEL_mean2=pd.DataFrame(np.mean(ARISEL,axis=0)).sort_values(by=0)
ARISEL=ARISEL.loc[ARISEL_mean.index,ARISEL_mean2.index]
ARISEL=ARISEL.loc[['.' not in c   for c in  ARISEL.index],:]
ARISEL.columns=ARISEL.columns.map(datasetdic)

In [8]:
plt.figure(figsize=(10,10))
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
sns.heatmap(ARISEL,cmap='Purples')
plt.savefig('../../figures/7.spatial_architecture/simulations_scores_by_dataset.pdf')

ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 720x720 with 0 Axes>

### We read and plot sample-specific additional information

In [ ]:
sampleinfo=pd.read_csv('../../figures/7.spatial_architecture/simulated_sample_info.csv',index_col=0)
mean_ARI=pd.DataFrame(np.mean(ARI.fillna(0),axis=0),columns=['mean_ARI'])
expe=dict(zip(mean_ARI.index,mean_ARI['mean_ARI']))
sampleinfo['mean_ARI']=sampleinfo.index.map(expe)
sampleinfo=sampleinfo[sampleinfo['mean_ARI']>0]
sampleinfo.index=sampleinfo.index.map(datasetdic)
sampleinfo=sampleinfo.loc[ARISEL.columns,:]

In [ ]:
correl=[]
for col in sampleinfo.columns:
    correl.append(np.corrcoef(sampleinfo[col],sampleinfo['mean_ARI'])[0,1])
sampleinfosub=sampleinfosub=sampleinfo.loc[:,sampleinfo.columns[0:5]]
sampleinfosub=sampleinfosub.div(sampleinfosub.max(axis=0),axis=1)

In [ ]:
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.figure(figsize=(2,10))
sns.heatmap(sampleinfosub,cmap='Reds')
plt.savefig('../../figures/7.spatial_architecture/sampleinfo_all.pdf')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
for col in sampleinfosub.columns:
    plt.figure(figsize=(2,10))
    sns.heatmap(sampleinfosub.loc[:,[col]],cmap='Reds')
    plt.savefig('../../figures/7.spatial_architecture/sampleinfo_'+str(col)+'.pdf')
    

# We also plot ARI normalizing by sample

In [ ]:
ARISELNORM=ARISEL.div(ARISEL.max(axis=0),axis=1)
ARISEL_ranked=ARISEL.rank(axis=0)
order=pd.DataFrame(np.median(ARISEL_ranked,axis=1),index=ARISEL_ranked.index).sort_values(by=0)
ARISEL_ranked_sorted=ARISEL_ranked.loc[order.index,:]

In [ ]:
plt.figure(figsize=(10,10))
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
sns.heatmap(ARISEL_ranked,cmap='Blues',vmin=300)
plt.savefig('../../figures/7.spatial_architecture/agreement_in_processing_workflows.pdf')

# Perturbation experiment, based on best workflow

Similar to what we do with real data in 1_6, we take the top performing workflow as reference and explore the effect of modifying individual parameters on the final outcome

In [ ]:
dat=[el.split('_') for el in ARISEL.index]
metadata=pd.DataFrame(dat)
metadata_nam=metadata.copy()
best_option=metadata.iloc[-1:,:] #### we select the best or not
for col in metadata.columns:
    metadata.loc[:,col]=~metadata.loc[:,col].isin(best_option.iloc[:,col])*1
ARICLOSE=ARISEL.loc[list(np.sum(metadata,axis=1)==1),:]
metadata_close=metadata.loc[list(np.sum(metadata,axis=1)==1),:]
metadata_namclose=metadata_nam.loc[list(np.sum(metadata,axis=1)==1),:]
newnam=[]
for ind in metadata_close.index:
    newnam.append(pd.DataFrame(metadata_namclose.loc[ind,list(metadata_close.loc[ind,:]==1)]).iloc[0,0])
ARICLOSE=pd.concat([ARICLOSE,ARISEL.iloc[-1:,:]])
newnam.append('best')
ARICLOSE.index=newnam
selected=pd.DataFrame(np.sum(metadata,axis=1)==1)
selected.loc[selected.iloc[:,0]==1,:].index
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.figure(figsize=(4,5))
sns.barplot(y=ARICLOSE.index,x=list(ARICLOSE.mean(axis=1)),palette='YlOrRd')
plt.savefig('../../figures/7.spatial_architecture/perturbation_test_simulated_barplot.pdf')

We do the same, but for the 2nd best workflow

In [ ]:
dat=[el.split('_') for el in ARISEL.index]
metadata=pd.DataFrame(dat)
metadata_nam=metadata.copy()
best_option=metadata.iloc[-2:-1,:] #### we select the best or not
for col in metadata.columns:
    metadata.loc[:,col]=~metadata.loc[:,col].isin(best_option.iloc[:,col])*1
ARICLOSE=ARISEL.loc[list(np.sum(metadata,axis=1)==1),:]
metadata_close=metadata.loc[list(np.sum(metadata,axis=1)==1),:]
metadata_namclose=metadata_nam.loc[list(np.sum(metadata,axis=1)==1),:]
newnam=[]
for ind in metadata_close.index:
    newnam.append(pd.DataFrame(metadata_namclose.loc[ind,list(metadata_close.loc[ind,:]==1)]).iloc[0,0])
ARICLOSE=pd.concat([ARICLOSE,ARISEL.iloc[-1:,:]])
newnam.append('2nd best')
ARICLOSE.index=newnam
selected=pd.DataFrame(np.sum(metadata,axis=1)==1)
selected.loc[selected.iloc[:,0]==1,:].index
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.figure(figsize=(4,5))
sns.barplot(y=ARICLOSE.index,x=list(ARICLOSE.mean(axis=1)),palette='YlOrRd')
plt.savefig('../../figures/7.spatial_architecture/perturbation_test_simulated_barplot_second_best_workflow.pdf')

## Next, we generate the plots regarding the preprocessing options of each method

In [ ]:
ALLSCORES=ALLSCORES.sort_values(by='ARI')
alld=[n.split('_') for n in ARISEL.index]
allso=pd.DataFrame(alld)
allso.columns=['Software','normalization', 'log', 'neighbors', 'pcs', 'target_sum','skip', 'svg','scale','skip2', 'clustering_alg', ]
allso['ARI_mean']=list(np.mean(ARISEL_ranked.iloc[:,:],axis=1))
allso=allso.sort_values(by='ARI_mean')
alsel=allso.loc[['.' not in c   for c in  allso['clustering_alg']],:]

In [ ]:
sns.heatmap(alsel.tail(20).loc[:,['ARI_mean']],cmap='autumn')
plt.savefig('../../figures/7.spatial_architecture/top20_ranks_heatmap.pdf')

In [ ]:
alsel['ARI_mean']=ARISEL.shape[0]-alsel['ARI_mean']#np.max(alsel['ARI_mean'])

In [ ]:
import matplotlib.patches as mpatches
plt.figure(figsize=(20,10))
plt.scatter(range(0,alsel.shape[0]),alsel['ARI_mean'],s=2)
fig, ax = plt.subplots(figsize=(20,7),nrows=len(alsel.columns[:-1]), ncols=1)
nun=0
for cols in alsel.columns[:-1]:
    import matplotlib.pyplot as plt
    import numpy as np
    import random
    import matplotlib
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['ps.fonttype'] = 42
#    print(cols)
    # Generate a list of 50 random hex colors
    hex_colors = ['#' + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(50)]

    # Convert hex colors to RGB in the range from 0 to 1 and store them in a matrix
    rgb_colors = []
    for hex_color in hex_colors:
        hex_color = hex_color.lstrip('#')
        rgb = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (0, 2, 4))
        rgb_colors.append(rgb)
    data=list(alsel.tail(25)[cols])

    colos=rgb_colors
    categories={}
    nu=0
    for cat in np.unique(data):
        categories[cat]=colos[nu]
        nu=nu+1

    colors = np.array([categories[category] for category in data])

    # Create a figure and axis

    # Plot the 1D color mesh using imshow
    ax[nun].imshow(colors.reshape(1, -1, 3),aspect='auto')
    plt.axis('off')
    patches =[mpatches.Patch(color=categories[i],label=i) for i in categories.keys()]
    plt.legend(handles=patches,loc=9)
    # Set the axis limits
    ax[nun].set_xlim(0, len(data))
    #ax.set_ylim(0, 800)

    # Add a colorbar for reference
    #cbar = plt.colorbar(plt.cm.ScalarMappable(cmap=None, norm=None), ax=ax, orientation='horizontal')
    plt.axis('off')
    nun=nun+1
    # Show the plot
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('../../figures/7.spatial_architecture/top_processing_workflows.pdf')
plt.show()


In [ ]:
alsel2=alsel.loc[:,['Software','normalization','target_sum','log','svg','skip2','pcs','neighbors','clustering_alg']]
alsel2.loc[alsel2['normalization']!='normTrue','target_sum']='NaN'
alsel2.loc[alsel2['normalization']=='normpearson','log']='NaN'
alsel2.loc[alsel2['normalization']=='normSCT','log']='NaN'
alsel2.loc[alsel2['normalization']=='normSCT','svg']='NaN'

In [ ]:
import matplotlib.patches as mpatches
plt.figure(figsize=(20,10))
plt.scatter(range(0,alsel.shape[0]),alsel['ARI_mean'],s=2)
fig, ax = plt.subplots(figsize=(20,7),nrows=len(alsel.columns[:-1]), ncols=1)

for cols in alsel2.columns[:]:
    import matplotlib.pyplot as plt
    import numpy as np
    import random
    import matplotlib
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['ps.fonttype'] = 42
#    print(cols)
    # Generate a list of 50 random hex colors
    hex_colors = ['#' + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(50)]
    fig, ax = plt.subplots(figsize=(20,0.5),nrows=1, ncols=1)
    nun=0
    # Convert hex colors to RGB in the range from 0 to 1 and store them in a matrix
    rgb_colors = []
    for hex_color in hex_colors:
        hex_color = hex_color.lstrip('#')
        rgb = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (0, 2, 4))
        rgb_colors.append(rgb)
    data=list(alsel2[cols])

    colos=rgb_colors
    categories={}
    nu=0
    for cat in np.unique(data):
        if cat=='NaN':
            categories[cat]=(0.5,0.5,0.5)
        else:
            categories[cat]=colos[nu]
        nu=nu+1

    colors = np.array([categories[category] for category in data])

    # Create a figure and axis

    # Plot the 1D color mesh using imshow
    ax.imshow(colors.reshape(1, -1, 3),aspect='auto')
    plt.axis('off')
    patches =[mpatches.Patch(color=categories[i],label=i) for i in categories.keys()]
    plt.legend(handles=patches,loc=1)
    # Set the axis limits
    ax.set_xlim(0, len(data))
    #ax.set_ylim(0, 800)

    # Add a colorbar for reference
    #cbar = plt.colorbar(plt.cm.ScalarMappable(cmap=None, norm=None), ax=ax, orientation='horizontal')
    plt.axis('off')
    nun=nun+1
    # Show the plot
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('../../figures/7.spatial_architecture/all_processing_workflows.pdf')
plt.show()


In [ ]:
import matplotlib.patches as mpatches
plt.figure(figsize=(20,10))
plt.scatter(range(0,alsel.shape[0]),alsel['ARI_mean'],s=2)
fig, ax = plt.subplots(figsize=(40,12),nrows=len(alsel.columns[:-1]), ncols=1)
nun=0
for cols in alsel2.columns[:]:
    matplotlib.rcParams['pdf.fonttype'] = 42
    matplotlib.rcParams['ps.fonttype'] = 42
    # Generate a list of 50 random hex colors
    hex_colors = ['#' + ''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(50)]
    # Convert hex colors to RGB in the range from 0 to 1 and store them in a matrix
    rgb_colors = []
    for hex_color in hex_colors:
        hex_color = hex_color.lstrip('#')
        rgb = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (0, 2, 4))
        rgb_colors.append(rgb)
    data=list(alsel2[cols])

    colos=rgb_colors
    categories={}
    nu=0
    for cat in np.unique(data):
        if cat=='NaN':
            categories[cat]=(0.8,0.8,0.8)
        else:
            categories[cat]=colos[nu]
        nu=nu+1
    colors = np.array([categories[category] for category in data])
    ax[nun].imshow(colors.reshape(1, -1, 3),aspect='auto')
    plt.axis('off')
    patches =[mpatches.Patch(color=categories[i],label=i) for i in categories.keys()]
    plt.legend(handles=patches,loc=9)
    # Set the axis limits
    ax[nun].set_xlim(0, len(data))
    # Add a colorbar for reference
    plt.axis('off')
    nun=nun+1
    # Show the plot
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig('../../figures/7.spatial_architecture/all_processing_workflows.pdf')
plt.show()
